## prompt test notebook

In [ ]:
import os
import yaml
import torch

from pathlib import Path
from langchain_community.vectorstores import Chroma
from langchain.chains import create_qa_with_sources_chain
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

from load_embedding import load_embedding
from set_db import create_chroma_db
from load_llm import load_model

# 캐시 디렉토리 설정
cache_dir = './weights'
os.makedirs(cache_dir, exist_ok=True)
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = os.path.join(cache_dir, 'datasets')
os.environ['HUGGINGFACE_HUB_CACHE'] = cache_dir
os.environ['TORCH_HOME'] = os.path.join(cache_dir, 'torch')


In [ ]:
class LegalQASystem:
    def __init__(self, prompt_file, model_name, cache_dir='./weights'):
        """prompt/qna.yaml, prompt/law.yaml"""
        self.load_prompt(prompt_file)
        self.setup_qa_system()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.llm = load_model('llama') # or qwen
    
    def load_prompt(self, prompt_file):
        with open(prompt_file, 'r', encoding='utf-8') as f:
            self.prompts = yaml.safe_load(f)
    
    def setup_qa_system(self):
        embeddings = load_embedding()

        ## Set DB
        self.db = create_chroma_db(embeddings)

        qa_template = PromptTemplate(
            input_variables=['context','chat_history','question'],
            template = self.prompts['qa_template']
        )

        self.memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=True,
        )

        self.qa_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.db.as_retriever(search_kwargs={'k' 5}),
            memory=self.memory,
            combine_docs_chain_kwargs={'prompt':qa_template},
            return_source_documents=True,
        )
    
    def get_answer(self, question):
        """질문에 대한 답변 생성"""
        try:
            result = self.qa_chain({'question': question})

            sources =[]
            for doc in result['source_documents']:
                citation = self.prompts['citation_template'].format(
                    law_name=doc.metadata.get('law_name', '정보 없음'),
                    paragraph=doc.metadata.get('paragraph', '정보 없음'),
                    article_number=doc.metadata.get('article_number', '정보 없음'),
                    effective_date=doc.metadata.get('effective_date', '정보 없음'),
                )
                sources.append(citation)
            
            return {
                'status' : 'success',
                'answer' : result['answer'],
                'sources' : sources
            }
        except Exception as e:
            return {
                'status' : 'error',
                'answer' : self.prompts['error_message'],
                'error' : str(e)
            }
        
if __name__ == '__main__':
    qa_system = LegalQASystem(prompt_file='prompt/qna.yaml', model_name = 'llama')

    question = '시각장애인 보조견 동반 출입 거부 시 처벌 규정이 있나요?'
    response = qa_system.get_answer(question)

    print('답변: ', response['answer'])
    print('\n참고문서:')
    for source in response['sources']:
        print(source)

## Template 읽어오기 

In [1]:
from langchain_core.prompts import load_prompt

prompt = load_prompt('prompt/law.yaml', encoding= 'utf-8')
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

### ChatPromptTemplate

- 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있다.
- 메세지는 튜플로 구성, (role, message) 형태로 구성한다.
- role
    - "system"
    - "human"
    - "ai"

In [6]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)
chat_template

ChatPromptTemplate(input_variables=['name', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='반가워요!'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='안녕하세요! 무엇을 도와드릴까요?'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})])

## MessagePlaceholder

또한 LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 `MessagePlaceholder` 를 제공합니다. 

메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용할 수 있습니다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

In [ ]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

## Load DB

- load embedding

In [ ]:
from load_embedding import load_embedding

# 모델 로드
embedding = load_embedding()

In [ ]:
from set_db import create_chroma_db

db = create_chroma_db()

In [2]:
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

def load_model(model_name):
    if model_name == 'llama':
        tokenizer = AutoTokenizer.from_pretrained(
            "davidkim205/Ko-Llama-3-8B-Instruct",
            cache_dir=cache_dir
            )
        model = AutoModelForCausalLM.from_pretrained(
            "davidkim205/Ko-Llama-3-8B-Instruct",
            device_map="auto",
            torch_dtype=torch.float16,
            cache_dir=cache_dir
            )
    elif model_name == 'qwen': 
        tokenizer = AutoTokenizer.from_pretrained(
            "davidkim205/Ko-Qwen-3-8B-Instruct",
            cache_dir=cache_dir
            )
        model = AutoModelForCausalLM.from_pretrained(
            "davidkim205/Ko-Qwen-3-8B-Instruct",
            device_map="cuda",
            torch_dtype=torch.float16,
            cache_dir=cache_dir
            )
        
    return tokenizer, model

def load_embedding(model_name, device):
    if model_name == 'bge':
        model_name = "upskyy/bge-m3-korean"
        embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={'device': device},
            encode_kwargs={'normalize_embeddings': True},
            cache_folder=cache_dir
        )
        return embeddings
    else:
        assert False, f"Unknown model name: {model_name}"


def load_doc(runpod):
    if runpod:
        pdf_path = "/workspace/LangEyE/crawling/장애인복지법.pdf"
        docs = LegalText(pdf_path).documents
    else:
        pdf_path = "/Volumes/MINDB/24년/SW아카데미/LangEyE/crawling/장애인복지법.pdf"
        docs = LegalText(pdf_path).documents
        
    return docs

In [3]:
embeddings = load_embedding('bge', device)
tokenizer, model = load_model('llama')
docs = load_doc(True)

/tmp/ipykernel_530/2991526531.py:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:


retriever = vectorstore.as_retriever(
    search_type="mmr",  # Maximum Marginal Relevance 사용
    search_kwargs={
        "k": 4,  # 검색 결과 수 증가
        "fetch_k": 20,  # candidate 검색 수 증가
        "lambda_mult": 0.7,  # diversity vs similarity 조절
        "filter": lambda x: True  # 기본 필터 설정
    }
)


pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.15
    )
llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cuda:0


In [9]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

def find_article(question: str) -> dict:
    """질문에서 조항 번호를 추출하고 해당 필터를 반환"""
    import re
    
    # 조항 번호 패턴 매칭 (예: 제59조의9, 제2조 등)
    pattern = r'제(\d+)조의?(\d+)?'
    match = re.search(pattern, question)
    
    if match:
        article_num = match.group(0)
        return {"article_number": article_num}
    return None

# 3. retrieval_chain 수정
retrieval_chain = RunnableParallel({
    "question": RunnablePassthrough(),
    "retrieved_docs": RunnableLambda(
        lambda x: format_docs(
            retriever.get_relevant_documents(
                x,
                search_kwargs={
                    "filter": find_article(x)
                } if find_article(x) else {}
            )
        )
    )
})

# 4. prompt 템플릿 수정
template = """장애인복지법 관련 질의응답을 진행하겠습니다.
[법령 기본 정보]
법률명: {law_title}
문서 유형: {document_type}
시행일자: {effective_date}
법률 분야: {legal_area}

[참고할 법령 내용]
{context}

[질문]
{question}

[답변 형식]
1. 먼저 해당 조항의 존재 여부를 명시해 주세요.
2. 조항이 존재하는 경우, 조항 번호와 전체 내용을 인용해 주세요.
3. 조항이 없는 경우, "해당 조항은 제시된 법령에 포함되어 있지 않습니다"라고 답변해 주세요.

[답변]"""

# 5. 실행 시 에러 처리 추가
def safe_invoke(question: str):
    try:
        return rag_chain.invoke(question)
    except Exception as e:
        return f"검색 중 오류가 발생했습니다. 해당 조항이 법령에 포함되어 있지 않을 수 있습니다. 오류: {str(e)}"

# 테스트
test_question = "장애인복지법 제59조의7에는 무엇이 명시되어 있나요?"
answer = safe_invoke(test_question)
print(f"질문: {test_question}")
print(f"답변: {answer}")

질문: 장애인복지법 제59조의7에는 무엇이 명시되어 있나요?
답변: 장애인복지법 관련 질의응답을 진행하겠습니다.
[법령 기본 정보]
법률명: 
문서 유형: 법률
시행일자: 2024-10-22
법률 분야: 장애인복지

[참고할 법령 내용]

            [법령 위치]
            법률명: 
            장: 제5장 복지시설과 단체
            조문: 제59조의6
            
            [조문 내용]
            장애인학대 및 장애인 대상 성범죄 신고인에 대하여는 「특정범죄
신고자 등 보호법」 제7조부터 제13조까지의 규정을 준용한다.
[본조신설 2017. 12. 19.]
[종전 제59조의6은 제59조의8로 이동 <2017. 12. 19.>]
        

---


            [법령 위치]
            법률명: 
            장: 제1장 총칙
            조문: 제3조
            
            [조문 내용]
            장애인복지의 기본이념은 장애인의 완전한 사회 참여와 평등을 통하여 사회통합을 이루는 데에 있다.
        

---


            [법령 위치]
            법률명: 
            장: 제3장 복지 조치
            조문: 제44조
            
            [조문 내용]
            국가, 지방자치단체 및 그 밖의 공공단체는 장애인복지시설과 장애인복지단체에서 생산한 물품의
우선 구매에 필요한 조치를 마련하여야 한다.
[전문개정 2012. 1. 26.]
 
제45조 삭제 <2017. 12. 19.>
 
제45조의2 삭제 <2017. 12. 19.>
        

---


            [법령 위치]
            법률명: 
            장: 제1장 총칙
            조문: 제1조
            
  